In [1]:
## import libraries
from stitch import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time, sys, math, zipfile, shutil, pickle, threading, multiprocessing

CSPICE_N0067


In [2]:
## set directory - desired perijove to be processed (make sure STITCHING/KERNELS contains proper kernels)
perijove = 20
os.chdir("../DATA/RAW/PERIJOVE-" + str(perijove))
print("Current perijove directory: " + os.getcwd())

Current perijove directory: /home/uzumochi/eigenjuno/DATA/RAW/PERIJOVE-20


In [3]:
## unzip junocam website data - just drop zip files in perijove directory and run this block!
for file in sorted(os.listdir('.')):
    meta_num = file.split('-')[0]
    if fnmatch.fnmatch(file, '*-Data.zip') and not os.path.isfile(meta_num + '-Metadata.json'):
        print('Unzipped', meta_num, 'image and data')
        with zipfile.ZipFile(file, 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                zip_ref.extract(n)
        with zipfile.ZipFile(meta_num + '-ImageSet.zip', 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                if 'raw' in n:
                    zip_ref.extract(n)
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*.zip') or fnmatch.fnmatch(file, '*Zone.Identifier'):
        os.remove(file)

if os.path.isdir('DataSet'):
    for file in sorted(os.listdir('DataSet')):
        shutil.move('DataSet/' + file, file)
    os.rmdir('DataSet')

if os.path.isdir('ImageSet'):
    for file in sorted(os.listdir('ImageSet')):
        shutil.move('ImageSet/' + file, file)
    os.rmdir('ImageSet')

In [4]:
## multithreaded execution - process all images in directory that have don't already have a stitched version
img_nums = []
for file in os.listdir('.'):
    if file.endswith('.json') and not os.path.isfile(file.split('-')[0] + '-Stitched.png'):
        img_nums.append(file.split('-')[0])
print_str = 'Processing the following images: '
for im in img_nums:
    print_str += (im + ', ')
print_str = print_str[:-2]
print(print_str)

num_cpus = multiprocessing.cpu_count() - 1
curr = 0

print("Begin stitching threads")

while curr < len(img_nums):
    threads = []
    num_threads = num_cpus if num_cpus < len(img_nums) else len(img_nums)
    print('Running ' + str(num_threads) + ' threads')
    for t in range(num_threads):
        try:
            threads.append(threading.Thread(target = stitch, args = (str(img_nums[curr]), )))
        except IndexError:
            print('\nStarting new round of threads')
        threads[t].start()
        curr += 1
    for t in threads:
        t.join()

print("End stitching threads")

Processing the following images
Begin stitching threads
End stitching threads


In [5]:
## copy all processed images and their metadata + surface rasters to testing directory
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*-Metadata.json') or fnmatch.fnmatch(file, '*-Raster.npy') \
        or fnmatch.fnmatch(file, '*-Stitched.png'):
        shutil.copy(file, '../../TEST/' + file)